In [1]:
# 0.1. Cài đặt các thư viện cần thiết
!pip install -q fpdf2 noisereduce librosa tensorflow scikit-learn matplotlib seaborn pytz PyDrive2 imbalanced-learn 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.7/72.7 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.7/251.7 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 56.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 109.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
category-encoders 2.7.0 requires scikit-learn<1.6.0,>=1.0.0, but you have scikit-learn 1.6.1 which is incompatible.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.


In [2]:
# CÀI ĐẶT & CẤU HÌNH 
# 0.2. Import thư viện

# --- Thư viện chuẩn của Python ---
import os
import glob 
import random
import datetime
import pytz
import shutil
from itertools import cycle
from tqdm import tqdm
import gc

# --- Thư viện xử lý dữ liệu và tính toán ---
import numpy as np
import pandas as pd
from imblearn.over_sampling import SMOTE

# --- Thư viện trực quan hóa ---
import seaborn as sns
import matplotlib.pyplot as plt

# --- Thư viện xử lý âm thanh ---
import librosa
import noisereduce as nr

# --- Thư viện Machine Learning (Scikit-learn) ---
from sklearn.preprocessing import LabelEncoder, label_binarize
from sklearn.model_selection import StratifiedGroupKFold
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc as sklearn_auc
from sklearn.utils import class_weight

# --- TensorFlow & Keras ---
import tensorflow as tf
from tensorflow.keras import mixed_precision
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.applications import EfficientNetV2B2
from tensorflow.keras.applications.efficientnet_v2 import preprocess_input
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.utils import register_keras_serializable

# Import tất cả các lớp (Layers) cần thiết cho cả CNN và CRNN
from tensorflow.keras.layers import (
    Input, 
    Dense, 
    Dropout, 
    GlobalAveragePooling2D, 
    GlobalMaxPooling2D,  
    Concatenate,          
    BatchNormalization, 
    Activation,
    Reshape,              # Thêm cho CRNN
    LSTM,                 # Thêm cho CRNN
    Bidirectional         # Thêm cho CRNN
)

# --- Thư viện khác ---
from fpdf import FPDF 

print(f"TensorFlow Version: {tf.__version__}")
print("Tất cả các thư viện đã được import thành công.")


try:
    # --- 1. ƯU TIÊN KẾT NỐI VỚI TPU ---
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()
    strategy = tf.distribute.TPUStrategy(tpu)
    policy = 'mixed_bfloat16'
    mixed_precision.set_global_policy(policy)
    print(" KẾT NỐI TPU THÀNH CÔNG!")
    print(f"   - Số lượng nhân (replicas): {strategy.num_replicas_in_sync}")
    print(f"   - Kiểu dữ liệu (DType Policy): {policy}")

except Exception:
    print(" Không tìm thấy TPU. Đang kiểm tra GPU...")
    
    # --- 2. SỬA LỖI: CHỈ SỬ DỤNG 1 GPU ĐỂ GỠ LỖI ---
    gpus = tf.config.list_physical_devices('GPU')
    if gpus:
        try:
            print(f"Tìm thấy {len(gpus)} GPU. Sẽ chỉ sử dụng GPU:0 để gỡ lỗi.")
            # Chỉ cho phép TensorFlow nhìn thấy GPU đầu tiên
            tf.config.set_visible_devices(gpus[0], 'GPU')
            
            # Sử dụng chiến lược mặc định, nó sẽ tự động chọn GPU duy nhất có sẵn
            strategy = tf.distribute.get_strategy()
            
            policy = 'mixed_float16'
            mixed_precision.set_global_policy(policy)
            
            print(" KẾT NỐI 1-GPU THÀNH CÔNG!")
            print(f"   - Số lượng nhân (replicas): {strategy.num_replicas_in_sync}")
            print(f"   - Kiểu dữ liệu (DType Policy): {policy}")

        except RuntimeError as e:
            # Lỗi này có thể xảy ra nếu visible devices đã được set trước đó
            print(e)
            print("Không thể thay đổi thiết bị GPU. Sử dụng chiến lược mặc định.")
            strategy = tf.distribute.get_strategy()

    else:
        # --- 3. NẾU KHÔNG CÓ GPU, SỬ DỤNG CPU ---
        print(" Không tìm thấy GPU. Sử dụng CPU.")
        strategy = tf.distribute.get_strategy()
        print(" Sử dụng chiến lược mặc định cho CPU.")
        print(f"   - Số lượng nhân (replicas): {strategy.num_replicas_in_sync}")


2025-10-14 15:10:25.402267: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1760454625.789462      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1760454625.913868      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


TensorFlow Version: 2.18.0
Tất cả các thư viện đã được import thành công.
 Không tìm thấy TPU. Đang kiểm tra GPU...
Tìm thấy 2 GPU. Sẽ chỉ sử dụng GPU:0 để gỡ lỗi.
 KẾT NỐI 1-GPU THÀNH CÔNG!
   - Số lượng nhân (replicas): 1
   - Kiểu dữ liệu (DType Policy): mixed_float16


In [3]:
# THIẾT LẬP CẤU HÌNH (PHIÊN BẢN CRNN)

# --- Cấu hình chung ---
SEED = 42
def set_seed(seed_value):
    os.environ['PYTHONHASHSEED'] = str(seed_value)
    random.seed(seed_value)
    np.random.seed(seed_value)
    tf.random.set_seed(seed_value)
set_seed(SEED)

KAGGLE_PROCESSED_DATA_PATH = "/kaggle/input/ngt-spectrogram-id/"
KAGGLE_OUTPUT_PATH = "/kaggle/working/output_results"
CHECKPOINT_PATH = "/kaggle/working/checkpoints"
TFRECORD_OUTPUT_PATH = "/kaggle/working/tfrecords"
os.makedirs(CHECKPOINT_PATH, exist_ok=True)
os.makedirs(KAGGLE_OUTPUT_PATH, exist_ok=True)
os.makedirs(TFRECORD_OUTPUT_PATH, exist_ok=True)

CLASSES_TO_TRAIN = ['covid', 'asthma', 'healthy', 'tuberculosis']
ALL_CLASSES = ['healthy', 'asthma', 'covid', 'tuberculosis']
N_SPLITS = 5
TEST_SPLIT_RATIO = 0.15
MODEL_ID = f'CRNN_EfficientNetB2_CV' # Cập nhật tên model

# --- Cấu hình Kiến trúc CRNN ---
# Số lượng units trong lớp LSTM. Giá trị lớn hơn giúp học các chuỗi phức tạp hơn.
LSTM_UNITS = 256 

# --- Cấu hình Data Augmentation & Regularization ---
# Giữ các giá trị đã được cân bằng từ lần trước
AUG_FREQ_MASKS = 2
AUG_TIME_MASKS = 2
AUG_FREQ_MASK_SIZE = 40
AUG_TIME_MASK_SIZE = 50
DROPOUT_RATE_1 = 0.5
DROPOUT_RATE_2 = 0.3
WEIGHT_DECAY = 1e-4
LABEL_SMOOTHING_VALUE = 0.1

# --- Các công tắc Bật/Tắt ---
USE_DATA_AUGMENTATION = True 
USE_FOCAL_LOSS = True        
USE_COSINE_DECAY_RESTARTS = True 

# --- Cấu hình pipeline dữ liệu ---
BATCH_SIZE = 64 # Giảm BATCH_SIZE vì CRNN tốn nhiều bộ nhớ hơn
GLOBAL_BATCH_SIZE = BATCH_SIZE * strategy.num_replicas_in_sync
SHUFFLE_BUFFER_SIZE = 2048
INPUT_SHAPE = (260, 260, 3)

# --- Cấu hình Huấn luyện (Quy trình Fine-tuning) ---
print("--- CÁC THAM SỐ HUẤN LUYỆN CHO CRNN ---")
STAGE1_HEAD_EPOCHS = 20
STAGE1_HEAD_LR = 5e-5
STAGE1_HEAD_PATIENCE = 5
print(f"Giai đoạn 1 (Head): {STAGE1_HEAD_EPOCHS} epochs, LR={STAGE1_HEAD_LR}")

STAGE1_FINETUNE_TOTAL_EPOCHS = 150
STAGE1_FINETUNE_LR_INITIAL = 1e-5 
STAGE1_FINETUNE_PATIENCE = 30
RESTART_CYCLE_1_EPOCHS = 25
print(f"Giai đoạn 2 (Finetune): Tối đa {STAGE1_FINETUNE_TOTAL_EPOCHS} epochs, LR ban đầu={STAGE1_FINETUNE_LR_INITIAL}")

# --- Các tham số khác ---
GAMMA = 2.0 
MIN_DELTA = 1e-4


--- CÁC THAM SỐ HUẤN LUYỆN ---
Giai đoạn 1A (Head): 20 epochs, LR=5e-05
Giai đoạn 1B (Finetune): Tối đa 175 epochs, LR ban đầu=1e-05
Giai đoạn 2 (SMOTE Head): 50 epochs, LR=1e-05


In [4]:
# KHỞI TẠO CÁC HÀM CẦN THIẾT 

def get_patient_id(filepath, class_name):
    filename = os.path.basename(filepath)
    if class_name.lower() in ['asthma', 'covid', 'healthy']:
        return filename.split('_')[0]
    elif class_name.lower() == 'tuberculosis':
        return '_'.join(filename.split('_')[:-1]).replace('.npy', '')
    else:
        return filename.split('_')[0]

@register_keras_serializable()
class MacroF1Score(tf.keras.metrics.Metric):
    """
    Lớp metric để tính toán Macro F1-Score một cách chính xác trên toàn bộ epoch.
    """
    def __init__(self, num_classes, name='f1_macro', **kwargs):
        super(MacroF1Score, self).__init__(name=name, **kwargs)
        self.num_classes = num_classes
        self.true_positives = self.add_weight(name='tp', shape=(num_classes,), initializer='zeros')
        self.false_positives = self.add_weight(name='fp', shape=(num_classes,), initializer='zeros')
        self.false_negatives = self.add_weight(name='fn', shape=(num_classes,), initializer='zeros')

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_pred_labels = tf.argmax(tf.nn.softmax(y_pred), axis=1)
        y_true_labels = tf.argmax(y_true, axis=1)
        cm = tf.math.confusion_matrix(y_true_labels, y_pred_labels, num_classes=self.num_classes, dtype=tf.float32)
        tp = tf.linalg.diag_part(cm)
        fp = tf.reduce_sum(cm, axis=0) - tp
        fn = tf.reduce_sum(cm, axis=1) - tp
        self.true_positives.assign_add(tp)
        self.false_positives.assign_add(fp)
        self.false_negatives.assign_add(fn)

    def result(self):
        precision = self.true_positives / (self.true_positives + self.false_positives + tf.keras.backend.epsilon())
        recall = self.true_positives / (self.true_positives + self.false_negatives + tf.keras.backend.epsilon())
        f1 = 2 * (precision * recall) / (precision + recall + tf.keras.backend.epsilon())
        macro_f1 = tf.reduce_mean(f1)
        return macro_f1

    def reset_state(self):
        self.true_positives.assign(tf.zeros(self.num_classes))
        self.false_positives.assign(tf.zeros(self.num_classes))
        self.false_negatives.assign(tf.zeros(self.num_classes))

    # Thêm phương thức get_config
    def get_config(self):
        config = super(MacroF1Score, self).get_config()
        config.update({'num_classes': self.num_classes})
        return config

    def reset_state(self):
        # Reset các biến trạng thái về 0 ở đầu mỗi epoch
        self.true_positives.assign(tf.zeros(self.num_classes))
        self.false_positives.assign(tf.zeros(self.num_classes))
        self.false_negatives.assign(tf.zeros(self.num_classes))
        
def parse_tfrecord_fn(example):
    """
    Hàm parse TFRecord hoàn chỉnh, giải quyết triệt để lỗi NaN bằng cách:
    1. Làm sạch các giá trị NaN/inf trong tensor đầu vào.
    2. Sử dụng phép toán an toàn (tf.maximum) để tránh chia cho 0.
    """
    # 1. Định nghĩa cấu trúc của TFRecord
    feature_description = {
        'image': tf.io.FixedLenFeature([], tf.string),
        'label': tf.io.FixedLenFeature([], tf.int64),
    }
    example = tf.io.parse_single_example(example, feature_description)
    
    # 2. Đọc và định hình lại tensor ảnh
    image = tf.io.parse_tensor(example['image'], out_type=tf.float32)
    # QUAN TRỌNG: Đảm bảo kích thước này khớp với dữ liệu gốc của bạn
    image.set_shape([256, 126]) 
    
    # 3. Thay đổi kích thước và thêm kênh màu
    image_3d = tf.stack([image, image, image], axis=-1)
    image_resized = tf.image.resize(image_3d, [INPUT_SHAPE[0], INPUT_SHAPE[1]])
    
    # 4. LÀM SẠCH DỮ LIỆU (Bước quan trọng nhất)
    # Thay thế bất kỳ giá trị NaN hoặc Inf nào bằng số 0.0
    image_sanitized = tf.where(tf.math.is_finite(image_resized), image_resized, 0.0)
    
    # 5. CHUẨN HÓA (SCALING) MỘT CÁCH AN TOÀN
    min_val = tf.reduce_min(image_sanitized)
    max_val = tf.reduce_max(image_sanitized)
    
    denominator = max_val - min_val
    
    # Sử dụng tf.maximum để đảm bảo mẫu số không bao giờ bằng 0.
    # Đây là cách an toàn để thực hiện 'if denominator == 0' trong đồ thị TensorFlow.
    denominator = tf.maximum(denominator, 1e-7)
        
    image_scaled_01 = (image_sanitized - min_val) / denominator
    image_scaled_255 = image_scaled_01 * 255.0
    
    # 6. Áp dụng tiền xử lý của model
    image_preprocessed = preprocess_input(image_scaled_255)
    
    # 7. Chuyển nhãn sang dạng one-hot
    label = tf.one_hot(tf.cast(example['label'], tf.int32), depth=len(ALL_CLASSES))
    
    return image_preprocessed, label

def spec_augment(spectrogram):
    """
    Sử dụng các tham số từ ô cấu hình để áp dụng SpecAugment.
    """
    spectrogram_aug = spectrogram
    freq_bins = tf.shape(spectrogram)[0]
    time_steps = tf.shape(spectrogram)[1]
    
    # Frequency Masking
    for _ in range(AUG_FREQ_MASKS):
        f = tf.random.uniform(shape=(), minval=0, maxval=AUG_FREQ_MASK_SIZE, dtype=tf.int32)
        f0 = tf.random.uniform(shape=(), minval=0, maxval=freq_bins - f, dtype=tf.int32)
        freq_mask_1d = tf.concat([tf.ones((f0,), dtype=spectrogram.dtype), tf.zeros((f,), dtype=spectrogram.dtype), tf.ones((freq_bins - f0 - f,), dtype=spectrogram.dtype)], axis=0)
        freq_mask_3d = tf.reshape(freq_mask_1d, (freq_bins, 1, 1))
        spectrogram_aug *= freq_mask_3d
        
    # Time Masking
    for _ in range(AUG_TIME_MASKS):
        t = tf.random.uniform(shape=(), minval=0, maxval=AUG_TIME_MASK_SIZE, dtype=tf.int32)
        t0 = tf.random.uniform(shape=(), minval=0, maxval=time_steps - t, dtype=tf.int32)
        time_mask_1d = tf.concat([tf.ones((t0,), dtype=spectrogram.dtype), tf.zeros((t,), dtype=spectrogram.dtype), tf.ones((time_steps - t0 - t,), dtype=spectrogram.dtype)], axis=0)
        time_mask_3d = tf.reshape(time_mask_1d, (1, time_steps, 1))
        spectrogram_aug *= time_mask_3d
        
    return spectrogram_aug

def augment(image, label):
    image = spec_augment(image)
    return image, label

# === KIẾN TRÚC MÔ HÌNH CRNN MỚI ===
@register_keras_serializable()
class FinalModelCRNN(tf.keras.Model):
    def __init__(self, input_shape_config, num_classes_config, lstm_units_config, **kwargs):
        super(FinalModelCRNN, self).__init__(**kwargs)
        self.input_shape_config = input_shape_config
        self.num_classes_config = num_classes_config
        self.lstm_units_config = lstm_units_config

        # 1. Phần CNN: Vẫn là EfficientNet làm bộ trích xuất đặc trưng
        self.base_model = EfficientNetV2B2(
            weights='imagenet',
            include_top=False, 
            input_shape=self.input_shape_config
        )
        # Output shape của base_model sẽ là (None, height, width, channels)
        # Ví dụ: (None, 8, 8, 1280)
        
        # 2. Lớp Reshape: Chuyển feature map thành chuỗi (sequence)
        # Ta sẽ coi chiều 'width' là các bước thời gian (timesteps)
        # Shape (None, 8, 8, 1280) -> (None, 8, 8 * 1280)
        output_shape = self.base_model.output_shape
        # target_shape = (timesteps, features_per_step)
        target_shape = (output_shape[1], output_shape[2] * output_shape[3]) 
        self.reshape = tf.keras.layers.Reshape(target_shape=target_shape, name="reshape_to_sequence")

        # 3. Phần RNN: Sử dụng Bidirectional LSTM để học cả hai chiều của chuỗi
        # Bidirectional làm tăng gấp đôi sức mạnh của LSTM
        self.lstm = tf.keras.layers.Bidirectional(
            tf.keras.layers.LSTM(self.lstm_units_config, return_sequences=False), # return_sequences=False vì chỉ cần output cuối cùng
            name="bidirectional_lstm"
        )
        
        # 4. Phần Phân loại (Head): Tương tự như trước
        self.dense1 = Dense(512, use_bias=False, kernel_regularizer=l2(WEIGHT_DECAY), name="dense_layer_1")
        self.bn1 = BatchNormalization(name="batch_norm_1")
        self.act1 = Activation('relu', name="activation_1")
        self.dropout1 = Dropout(DROPOUT_RATE_1, name="dropout_layer_1")
        
        self.dense_output = Dense(self.num_classes_config, activation='linear', dtype='float32', name="output_layer")

    def call(self, inputs, training=None):
        # 1. Trích xuất feature map bằng CNN
        x = self.base_model(inputs, training=training)
        
        # 2. Chuyển đổi thành chuỗi
        x = self.reshape(x)
        
        # 3. Xử lý chuỗi bằng LSTM
        x = self.lstm(x, training=training)
        
        # 4. Phân loại
        x = self.dense1(x)
        x = self.bn1(x, training=training)
        x = self.act1(x)
        x = self.dropout1(x, training=training)
        
        outputs = self.dense_output(x)
        return outputs

    def get_config(self):
        config = super(FinalModelCRNN, self).get_config()
        config.update({
            'input_shape_config': self.input_shape_config,
            'num_classes_config': self.num_classes_config,
            'lstm_units_config': self.lstm_units_config,
        })
        return config

    @classmethod
    def from_config(cls, config):
        return cls(**config)

        
def load_data_from_df(df):
    X, y = [], []
    for _, row in df.iterrows():
        X.append(np.load(row['filepath']))
        y.append(row['label'])
    return np.array(X), np.array(y)

def get_grad_cam_final(model, img_array, last_conv_layer_name, pred_index=None):
    """
    Tạo Grad-CAM cho một subclassed model.
    Lưu ý: Model phải được build (chạy qua dữ liệu một lần) trước khi gọi hàm này.
    """
    # Tạo một model trung gian với input là input của model chính,
    # và output là lớp conv cuối và output cuối cùng của model chính.
    grad_model = Model(
        inputs=model.inputs,
        outputs=[model.base_model.get_layer(last_conv_layer_name).output, model.output]
    )

    # Tính toán gradient
    with tf.GradientTape() as tape:
        # Đưa ảnh vào grad_model để lấy 2 output đã định nghĩa ở trên
        last_conv_layer_output, preds = grad_model(img_array)
        
        if pred_index is None:
            pred_index = tf.argmax(preds[0])
        class_channel = preds[:, pred_index]

    # Lấy gradient của lớp được dự đoán đối với feature map của lớp conv cuối
    grads = tape.gradient(class_channel, last_conv_layer_output)

    # Tính trung bình gradient và tạo heatmap
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
    last_conv_layer_output = last_conv_layer_output[0]
    heatmap = last_conv_layer_output @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)
    
    heatmap = tf.maximum(heatmap, 0) / (tf.math.reduce_max(heatmap) + tf.keras.backend.epsilon())
    
    return heatmap.numpy()

def overlay_grad_cam(spec, heatmap, alpha=0.6):
    heatmap_resized = tf.image.resize(heatmap[..., np.newaxis], (spec.shape[0], spec.shape[1]))
    heatmap_resized = np.uint8(255 * heatmap_resized)
    jet = plt.cm.get_cmap("jet")
    jet_colors = jet(np.arange(256))[:, :3]
    jet_heatmap = jet_colors[heatmap_resized.squeeze()]
    spec_display = np.stack([spec]*3, axis=-1)
    spec_display = (spec_display - spec_display.min()) / (spec_display.max() - spec_display.min())
    superimposed_img = jet_heatmap * alpha + spec_display
    superimposed_img = np.clip(superimposed_img, 0, 1)
    return superimposed_img

def _bytes_feature(value):
    """Returns a bytes_list from a string / byte."""
    if isinstance(value, type(tf.constant(0))):
        value = value.numpy()
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _int64_feature(value):
    """Returns an int64_list from a bool / enum / int / uint."""
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def serialize_example(image, label):
    """Creates a tf.train.Example message ready to be written to a file."""
    feature = {
        'image': _bytes_feature(tf.io.serialize_tensor(image)),
        'label': _int64_feature(label)
    }
    example_proto = tf.train.Example(features=tf.train.Features(feature=feature))
    return example_proto.SerializeToString()
class LearningRateLogger(tf.keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs=None):
        current_lr = tf.keras.backend.get_value(self.model.optimizer.learning_rate)
        print(f"\nEpoch {epoch+1}: Learning Rate is {current_lr:.2e}")

# CHUẨN BỊ DỮ LIỆU VÀ TẠO TFRECORD 

suspicious_files_to_remove = [
    '/kaggle/input/ngt-spectrogram-id/healthy/P0030101_123370_Dkwg3F7jMGaR7kbc-seg2.npy',
    '/kaggle/input/ngt-spectrogram-id/covid/P0032202_15897_PcbyJQWemBfghUYp-seg2.npy',
    '/kaggle/input/ngt-spectrogram-id/covid/P0027142_5701_hupBI5CxKMNCfe8b-seg1.npy',
    '/kaggle/input/ngt-spectrogram-id/covid/P0056214_89533_0WsmNRSKuQFGodg1-seg1.npy',
]
# --- BƯỚC 1: TẢI VÀ PHÂN CHIA DỮ LIỆU BAN ĐẦU ---
print("Bắt đầu chuẩn bị và phân chia dữ liệu...")
all_files_to_split = []
for class_name in ALL_CLASSES:
    source_dir = os.path.join(KAGGLE_PROCESSED_DATA_PATH, class_name)
    if os.path.exists(source_dir):
        files = glob.glob(os.path.join(source_dir, '*.npy'))
        for f in files:
            all_files_to_split.append({'filepath': f, 'label': class_name})

all_data_df = pd.DataFrame(all_files_to_split)
all_data_df['patient_id'] = all_data_df.apply(lambda row: get_patient_id(row['filepath'], row['label']), axis=1)

print(f"Số lượng mẫu ban đầu: {len(all_data_df)}")
all_data_df = all_data_df[~all_data_df['filepath'].isin(suspicious_files_to_remove)].reset_index(drop=True)
print(f"Số lượng mẫu sau khi lọc bỏ file 'im lặng': {len(all_data_df)}")

print("Tách tập Test cuối cùng (Hold-out set)...")
patient_ids = all_data_df['patient_id'].unique()
np.random.shuffle(patient_ids)
test_patient_count = int(len(patient_ids) * TEST_SPLIT_RATIO)
test_patients = patient_ids[:test_patient_count]
train_val_patients = patient_ids[test_patient_count:]

test_df = all_data_df[all_data_df['patient_id'].isin(test_patients)].reset_index(drop=True)
train_val_df = all_data_df[all_data_df['patient_id'].isin(train_val_patients)].reset_index(drop=True)

print(f"Đã tách: {len(train_val_df)} mẫu cho Train/Validation (CV) và {len(test_df)} mẫu cho Test cuối cùng.")

# --- BƯỚC 2: KHỞI TẠO LABEL ENCODER ---
le = LabelEncoder().fit(ALL_CLASSES)

# --- BƯỚC 3: CHỈ TẠO FILE TFRECORD CHO TẬP TEST ---
TFRECORD_OUTPUT_PATH = "/kaggle/working/tfrecords"
os.makedirs(TFRECORD_OUTPUT_PATH, exist_ok=True)
print(f"Bắt đầu chuyển đổi dữ liệu sang TFRecord tại: {TFRECORD_OUTPUT_PATH}")

print("--- Đang xử lý và tạo file cho tập test ---")
test_tfrecord_path = os.path.join(TFRECORD_OUTPUT_PATH, "test.tfrec")
with tf.io.TFRecordWriter(test_tfrecord_path) as writer:
    for _, row in tqdm(test_df.iterrows(), total=len(test_df), desc="Creating test.tfrec"):
        spectrogram = np.load(row['filepath']).astype(np.float32)
        
        image_tensor = tf.convert_to_tensor(spectrogram)
        image_3d = tf.stack([image_tensor]*3, axis=-1)
        image_resized = tf.image.resize(image_3d, [INPUT_SHAPE[0], INPUT_SHAPE[1]])
        min_val = tf.reduce_min(image_resized)
        max_val = tf.reduce_max(image_resized)
        image_scaled_01 = (image_resized - min_val) / (max_val - min_val + 1e-7)
        image_to_serialize = image_scaled_01 * 255.0

        label_encoded = le.transform([row['label']])[0]
        example = serialize_example(image_to_serialize, label_encoded)
        writer.write(example)

print("\\nChuẩn bị dữ liệu ban đầu hoàn tất!")

Bắt đầu chuẩn bị và phân chia dữ liệu...
Số lượng mẫu ban đầu: 33084
Số lượng mẫu sau khi lọc bỏ file 'im lặng': 33080
Tách tập Test cuối cùng (Hold-out set)...
Đã tách: 28060 mẫu cho Train/Validation (CV) và 5020 mẫu cho Test cuối cùng.
Bắt đầu chuyển đổi dữ liệu sang TFRecord tại: /kaggle/working/tfrecords
--- Đang xử lý và tạo file cho tập test ---


Creating test.tfrec: 100%|██████████| 5020/5020 [01:20<00:00, 62.29it/s]

\nChuẩn bị dữ liệu ban đầu hoàn tất!


In [5]:
# --- Ô CODE HUẤN LUYỆN ---

# === HÀM HỖ TRỢ VẼ BIỂU ĐỒ (giữ nguyên) ===
def plot_training_history(history, title_prefix, fold_number, output_dir):
    # (Nội dung hàm không thay đổi, giữ nguyên như trước)
    plt.figure(figsize=(18, 6))
    full_title = f"{title_prefix} - Fold {fold_number}"
    plt.suptitle(full_title, fontsize=16)
    plt.subplot(1, 2, 1)
    metrics_to_plot = ['accuracy', 'auc', 'f1_macro']
    colors = {'accuracy': 'blue', 'auc': 'green', 'f1_macro': 'red'}
    for metric in metrics_to_plot:
        if history.history.get(metric):
            plt.plot(history.history[metric], label=f'Training {metric.capitalize()}', color=colors[metric], linestyle='-')
        val_metric = f'val_{metric}'
        if history.history.get(val_metric):
            plt.plot(history.history[val_metric], label=f'Validation {metric.capitalize()}', color=colors[metric], linestyle='--')
    plt.title('Biểu đồ các chỉ số'); plt.xlabel('Epoch'); plt.ylabel('Giá trị')
    if any(metric in history.history for metric in metrics_to_plot):
        plt.legend(loc='lower left')
    plt.grid(True)
    plt.subplot(1, 2, 2)
    if history.history.get('loss'):
        plt.plot(history.history['loss'], label='Training Loss', color='orange')
    if history.history.get('val_loss'):
        plt.plot(history.history['val_loss'], label='Validation Loss', color='purple')
    plt.title('Biểu đồ Loss'); plt.xlabel('Epoch'); plt.ylabel('Loss')
    if history.history.get('loss') or history.history.get('val_loss'):
        plt.legend(loc='upper right')
    plt.grid(True)
    filename_prefix = title_prefix.replace(' ', '_').replace('-', '_').replace(':', '')
    filename = f"fold_{fold_number}_{filename_prefix}_metrics.png"
    filepath = os.path.join(output_dir, filename)
    plt.savefig(filepath)
    plt.close()
    print(f"Đã lưu biểu đồ: {os.path.basename(filepath)}")

# --- BƯỚC 1: KHỞI TẠO CÁC BIẾN CẦN THIẾT ---
print("Đang khởi tạo các biến cho Cross-Validation...")
AUTOTUNE = tf.data.AUTOTUNE
skf = StratifiedGroupKFold(n_splits=N_SPLITS, shuffle=True, random_state=SEED)
y_labels_for_split = le.transform(train_val_df['label'])
groups_for_split = train_val_df['patient_id'].values
fold_accuracies, fold_losses, fold_aucs, fold_f1s = [], [], [], []

if USE_FOCAL_LOSS:
    print("Đang tính toán trọng số alpha cho Focal Loss...")
    class_weights_array = class_weight.compute_class_weight('balanced', classes=np.unique(y_labels_for_split), y=y_labels_for_split)
    alpha_weights_list = class_weights_array.tolist()
    for i, w in enumerate(alpha_weights_list):
        class_name = le.inverse_transform([i])[0]
        print(f"- Lớp '{class_name}': {w:.2f}")
else:
    alpha_weights_list = []

# --- BƯỚC 2: BẮT ĐẦU VÒNG LẶP CROSS-VALIDATION ---
for fold, (train_indices, val_indices) in enumerate(skf.split(train_val_df, y_labels_for_split, groups_for_split)):
    fold_number = fold + 1
    print("-" * 60 + f"\nBắt đầu Fold {fold_number}/{N_SPLITS}\n" + "-" * 60)

    train_fold_df = train_val_df.iloc[train_indices]
    val_fold_df = train_val_df.iloc[val_indices]

    # === QUY TRÌNH HUẤN LUYỆN FINE-TUNING CHO CRNN ===
    print("\n" + "#" * 20 + f" HUẤN LUYỆN CRNN (Fold {fold_number}) " + "#" * 20)
    
    # --- 1A: Tạo pipeline dữ liệu ---
    train_tfrec_path = os.path.join(TFRECORD_OUTPUT_PATH, f"train_fold_{fold_number}.tfrec")
    val_tfrec_path = os.path.join(TFRECORD_OUTPUT_PATH, f"val_fold_{fold_number}.tfrec")
    def write_raw_tfrecord(df, path, desc):
        with tf.io.TFRecordWriter(path) as writer:
            for _, row in tqdm(df.iterrows(), total=len(df), desc=desc):
                spectrogram = np.load(row['filepath']).astype(np.float32)
                label_encoded = le.transform([row['label']])[0]
                example = serialize_example(spectrogram, label_encoded)
                writer.write(example)
    if not os.path.exists(train_tfrec_path):
        write_raw_tfrecord(train_fold_df, train_tfrec_path, f"Writing Raw Train Fold {fold_number}")
    if not os.path.exists(val_tfrec_path):
        write_raw_tfrecord(val_fold_df, val_tfrec_path, f"Writing Raw Val Fold {fold_number}")

    train_ds = tf.data.TFRecordDataset(train_tfrec_path).map(parse_tfrecord_fn, num_parallel_calls=AUTOTUNE)
    val_ds = tf.data.TFRecordDataset(val_tfrec_path).map(parse_tfrecord_fn, num_parallel_calls=AUTOTUNE).cache().batch(GLOBAL_BATCH_SIZE).prefetch(AUTOTUNE)
    
    if USE_DATA_AUGMENTATION:
        train_ds = train_ds.map(augment, num_parallel_calls=AUTOTUNE)
        
    train_ds = train_ds.shuffle(SHUFFLE_BUFFER_SIZE).repeat().batch(GLOBAL_BATCH_SIZE, drop_remainder=True).prefetch(AUTOTUNE)
    
    steps_per_epoch = len(train_indices) // GLOBAL_BATCH_SIZE
    validation_steps = len(val_indices) // GLOBAL_BATCH_SIZE
    
    # --- 1B: Huấn luyện 2 bước trên mô hình CRNN ---
    with strategy.scope():
        # Gọi kiến trúc CRNN mới
        model = FinalModelCRNN(
            input_shape_config=INPUT_SHAPE, 
            num_classes_config=len(ALL_CLASSES),
            lstm_units_config=LSTM_UNITS 
        )
        
        if USE_FOCAL_LOSS:
            loss_function = tf.keras.losses.CategoricalFocalCrossentropy(from_logits=True, alpha=alpha_weights_list, gamma=GAMMA, label_smoothing=LABEL_SMOOTHING_VALUE)
        else:
            loss_function = tf.keras.losses.CategoricalCrossentropy(from_logits=True, label_smoothing=LABEL_SMOOTHING_VALUE)
            
        # Giai đoạn 1: Huấn luyện Head (LSTM + Dense)
        print("\n--- Giai đoạn 1: Huấn luyện Head (LSTM + Dense) ---")
        model.base_model.trainable = False
        optimizer_head = tf.keras.optimizers.AdamW(learning_rate=STAGE1_HEAD_LR, weight_decay=WEIGHT_DECAY, epsilon=1e-7)
        model.compile(optimizer=optimizer_head, loss=loss_function, metrics=['accuracy'])
        history_head = model.fit(train_ds, validation_data=val_ds, epochs=STAGE1_HEAD_EPOCHS, steps_per_epoch=steps_per_epoch, validation_steps=validation_steps, callbacks=[EarlyStopping(monitor='val_loss', patience=STAGE1_HEAD_PATIENCE, restore_best_weights=True)], verbose=1)
        
        # Giai đoạn 2: Huấn luyện chính (Fine-tuning toàn bộ CRNN)
        print("\n--- Giai đoạn 2: Huấn luyện chính (Fine-tuning toàn bộ CRNN) ---")
        model.base_model.trainable = True
        
        if USE_COSINE_DECAY_RESTARTS:
            print("Sử dụng scheduler: CosineDecayRestarts")
            first_decay_steps = RESTART_CYCLE_1_EPOCHS * steps_per_epoch
            lr_scheduler = tf.keras.optimizers.schedules.CosineDecayRestarts(initial_learning_rate=STAGE1_FINETUNE_LR_INITIAL, first_decay_steps=first_decay_steps, t_mul=2.0, m_mul=0.9, alpha=0.1)
            optimizer_finetune = tf.keras.optimizers.AdamW(learning_rate=lr_scheduler, weight_decay=WEIGHT_DECAY, epsilon=1e-7)
            callbacks = [EarlyStopping(monitor='val_f1_macro', mode='max', patience=STAGE1_FINETUNE_PATIENCE, restore_best_weights=True, min_delta=MIN_DELTA, verbose=1), LearningRateLogger()]
        else:
            print("Không sử dụng CosineDecayRestarts")
            optimizer_finetune = tf.keras.optimizers.AdamW(learning_rate=STAGE1_FINETUNE_LR_INITIAL, weight_decay=WEIGHT_DECAY, epsilon=1e-7)
            callbacks = [EarlyStopping(monitor='val_f1_macro', mode='max', patience=STAGE1_FINETUNE_PATIENCE, restore_best_weights=True, verbose=1)]
        
        f1_macro = MacroF1Score(num_classes=len(ALL_CLASSES))
        model.compile(optimizer=optimizer_finetune, loss=loss_function, metrics=['accuracy', tf.keras.metrics.AUC(name='auc'), f1_macro])
        
        history_finetune = model.fit(train_ds, validation_data=val_ds, epochs=STAGE1_FINETUNE_TOTAL_EPOCHS, steps_per_epoch=steps_per_epoch, validation_steps=validation_steps, callbacks=callbacks, verbose=1)
    
    # === BƯỚC CUỐI: LƯU, VẼ BIỂU ĐỒ, ĐÁNH GIÁ, DỌN DẸP ===
    model_save_path = os.path.join(KAGGLE_OUTPUT_PATH, f'{MODEL_ID}_fold_{fold_number}.keras')
    model.save(model_save_path)
    print(f"\nĐã lưu model cuối cùng cho Fold {fold_number} tại: {model_save_path}")

    print("\n--- Vẽ và lưu các biểu đồ huấn luyện ---")
    plot_training_history(history_head, "Giai doan 1 - CRNN Head Training", fold_number, KAGGLE_OUTPUT_PATH)
    plot_training_history(history_finetune, "Giai doan 2 - CRNN Fine-tuning", fold_number, KAGGLE_OUTPUT_PATH)

    print("\n--- Đánh giá trên tập Validation và dọn dẹp ---")
    with strategy.scope():
        # Tải lại model đã lưu để đảm bảo tính nhất quán
        model_to_evaluate = tf.keras.models.load_model(model_save_path, custom_objects={'FinalModelCRNN': FinalModelCRNN, 'MacroF1Score': MacroF1Score})
        model_to_evaluate.compile(loss=loss_function, metrics=['accuracy', tf.keras.metrics.AUC(name='auc'), f1_macro])
    
    val_results = model_to_evaluate.evaluate(val_ds, verbose=0, return_dict=True)
    loss, accuracy, auc, f1 = val_results.get('loss', 0), val_results.get('accuracy', 0), val_results.get('auc', 0), val_results.get('f1_macro', 0)
    print(f"Fold {fold_number} - Validation Loss: {loss:.4f}, Accuracy: {accuracy:.4f}, AUC: {auc:.4f}, F1-Macro: {f1:.4f}")
    
    fold_accuracies.append(accuracy)
    fold_losses.append(loss)
    fold_aucs.append(auc)
    fold_f1s.append(f1)
    
    print("=" * 60 + "\nKết quả Cross-Validation Tạm thời:\n" 
          + f"  - Accuracy trung bình: {np.mean(fold_accuracies):.4f} +/- {np.std(fold_accuracies):.4f}\n"
          + f"  - Loss trung bình: {np.mean(fold_losses):.4f} +/- {np.std(fold_losses):.4f}\n"
          + f"  - AUC trung bình: {np.mean(fold_aucs):.4f} +/- {np.std(fold_aucs):.4f}\n"
          + f"  - F1-Macro trung bình: {np.mean(fold_f1s):.4f} +/- {np.std(fold_f1s):.4f}\n"
          + "=" * 60)
          
    print("\n--- Dọn dẹp file TFRecord tạm thời ---")
    try:
        files_to_remove = glob.glob(os.path.join(TFRECORD_OUTPUT_PATH, f"*fold_{fold_number}*"))
        for f in files_to_remove:
            os.remove(f)
            print(f"Đã xóa: {os.path.basename(f)}")
    except OSError as e:
        print(f"Lỗi khi xóa file: {e}")

    print("\n--- Dọn dẹp bộ nhớ RAM và đồ thị Keras ---")
    try:
        del model, model_to_evaluate, train_ds, val_ds
        tf.keras.backend.clear_session()
        gc.collect()
        print("Đã dọn dẹp bộ nhớ thành công.")
    except NameError as e:
        print(f"Một số biến có thể đã được dọn dẹp, bỏ qua lỗi: {e}")

# --- IN KẾT QUẢ TỔNG KẾT CUỐI CÙNG ---
print("\n\n" + "=" * 60 + "\nKẾT QUẢ CROSS-VALIDATION CUỐI CÙNG:\n" 
      + f"  - Validation Accuracy trung bình: {np.mean(fold_accuracies):.4f} +/- {np.std(fold_accuracies):.4f}\n"
      + f"  - Validation Loss trung bình: {np.mean(fold_losses):.4f} +/- {np.std(fold_losses):.4f}\n"
      + f"  - Validation AUC trung bình: {np.mean(fold_aucs):.4f} +/- {np.std(fold_aucs):.4f}\n"
      + f"  - Validation F1-Macro trung bình: {np.mean(fold_f1s):.4f} +/- {np.std(fold_f1s):.4f}\n"
      + "=" * 60)


Đang khởi tạo các biến cho Cross-Validation...
Đang tính toán trọng số alpha cho Focal Loss...
- Lớp 'asthma': 1.82
- Lớp 'covid': 0.80
- Lớp 'healthy': 0.87
- Lớp 'tuberculosis': 0.96
------------------------------------------------------------
Bắt đầu Fold 1/5
------------------------------------------------------------

#################### GIAI ĐOẠN 1: FINE-TUNING TRÊN DỮ LIỆU GỐC (Fold 1) ####################


Writing Raw Val Fold 1: 100%|██████████| 5607/5607 [01:12<00:00, 77.20it/s]


35839040/35839040 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step

--- Giai đoạn 1A: Huấn luyện Head trên dữ liệu gốc ---
Epoch 1/20


I0000 00:00:1760455123.170494      72 service.cc:148] XLA service 0x7f66b001a9e0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1760455123.173400      72 service.cc:156]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1760455126.161185      72 cuda_dnn.cc:529] Loaded cuDNN version 90300
2025-10-14 15:18:57.360760: E external/local_xla/xla/service/slow_operation_alarm.cc:65] Trying algorithm eng0{} for conv (f16[128,33,33,224]{3,2,1,0}, u8[0]{0}) custom-call(f16[128,33,33,56]{3,2,1,0}, f16[224,3,3,56]{3,2,1,0}), window={size=3x3 pad=1_1x1_1}, dim_labels=b01f_o01i->b01f, custom_call_target="__cudnn$convForward", backend_config={"cudnn_conv_backend_config":{"activation_mode":"kNone","conv_result_scale":1,"leakyrelu_alpha":0,"side_input_scale":0},"force_earliest_schedule":false,"operation_queue_id":"0","wait_on_operation_queues":[]} is taking a while...
2025-10-14 15:18:57.372258: E external/local_xla/xla/servic

  1/175 ━━━━━━━━━━━━━━━━━━━━ 2:59:42 62s/step - accuracy: 0.2344 - loss: 1.4372

...


--- Bước 2B: Áp dụng SMOTE ---

--- Bước 2C: Huấn luyện lại các lớp Dense trên dữ liệu SMOTE ---
Epoch 1/50
222/222 ━━━━━━━━━━━━━━━━━━━━ 9s 16ms/step - accuracy: 0.3076 - loss: 0.5332
Epoch 2/50
222/222 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.2892 - loss: 0.5371
Epoch 3/50
222/222 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.2911 - loss: 0.5244
Epoch 4/50
222/222 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.2909 - loss: 0.5162
Epoch 5/50
222/222 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.2959 - loss: 0.5106
Epoch 6/50
222/222 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.3045 - loss: 0.5026
Epoch 7/50
222/222 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.2978 - loss: 0.4948
Epoch 8/50
222/222 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.2991 - loss: 0.4929
Epoch 9/50
222/222 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.3027 - loss: 0.4840
Epoch 10/50
222/222 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.3076 - loss: 0.4746
Epoch 11/50
222/222 ━━━━━━━━━━━━━━━━━━━━

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/epoch_iterator.py:151: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


Fold 2 - Validation Loss: 0.8342, Accuracy: 0.4091, AUC: 0.7225, F1-Macro: 0.3703
Kết quả Cross-Validation Tạm thời:
  - Accuracy trung bình: 0.4127 +/- 0.0036
  - Loss trung bình: 0.8276 +/- 0.0066
  - AUC trung bình: 0.7109 +/- 0.0115
  - F1-Macro trung bình: 0.3641 +/- 0.0062

--- Dọn dẹp file TFRecord tạm thời ---
Đã xóa: train_fold_2.tfrec
Đã xóa: val_fold_2.tfrec

--- Dọn dẹp bộ nhớ RAM và đồ thị Keras ---
Một số biến đã được dọn dẹp trước đó, bỏ qua.
------------------------------------------------------------
Bắt đầu Fold 3/5
------------------------------------------------------------

#################### GIAI ĐOẠN 1: FINE-TUNING TRÊN DỮ LIỆU GỐC (Fold 3) ####################


Writing Raw Val Fold 3: 100%|██████████| 5628/5628 [00:44<00:00, 125.80it/s]



--- Giai đoạn 1A: Huấn luyện Head trên dữ liệu gốc ---
Epoch 1/20
175/175 ━━━━━━━━━━━━━━━━━━━━ 128s 425ms/step - accuracy: 0.2953 - loss: 1.4959 - val_accuracy: 0.3199 - val_loss: 1.1744
Epoch 2/20
175/175 ━━━━━━━━━━━━━━━━━━━━ 56s 320ms/step - accuracy: 0.2603 - loss: 1.5076 - val_accuracy: 0.4211 - val_loss: 0.9782
Epoch 3/20
175/175 ━━━━━━━━━━━━━━━━━━━━ 56s 319ms/step - accuracy: 0.2476 - loss: 1.5043 - val_accuracy: 0.4484 - val_loss: 0.9444
Epoch 4/20
175/175 ━━━━━━━━━━━━━━━━━━━━ 56s 322ms/step - accuracy: 0.2396 - loss: 1.4830 - val_accuracy: 0.4617 - val_loss: 0.9345
Epoch 5/20
175/175 ━━━━━━━━━━━━━━━━━━━━ 56s 322ms/step - accuracy: 0.2389 - loss: 1.4586 - val_accuracy: 0.4704 - val_loss: 0.9163
Epoch 6/20
175/175 ━━━━━━━━━━━━━━━━━━━━ 55s 317ms/step - accuracy: 0.2436 - loss: 1.4352 - val_accuracy: 0.4836 - val_loss: 0.9225
Epoch 7/20
175/175 ━━━━━━━━━━━━━━━━━━━━ 82s 469ms/step - accuracy: 0.2405 - loss: 1.4202 - val_accuracy: 0.4915 - val_loss: 0.9267
Epoch 8/20
175/175 ━━━━━━━

In [ ]:
# --- Bắt đầu quy trình đánh giá tổng hợp 5-Fold trên tập test ---

print("--- Chuẩn bị dữ liệu Test cho việc đánh giá ---")

# 1. Tạo Test Dataset từ file TFRecord đã được xử lý trước
TEST_TFREC_PATH = os.path.join(TFRECORD_OUTPUT_PATH, "test.tfrec")
if not os.path.exists(TEST_TFREC_PATH):
    print(f"Lỗi: Không tìm thấy file test.tfrec tại {TEST_TFREC_PATH}. Vui lòng chạy lại ô chuẩn bị dữ liệu.")
else:
    # Tạo dataset đã được batch để đưa vào model.evaluate
    test_ds_batched = tf.data.TFRecordDataset(TEST_TFREC_PATH)
    test_ds_batched = test_ds_batched.map(parse_tfrecord_fn, num_parallel_calls=AUTOTUNE)
    test_ds_batched = test_ds_batched.batch(GLOBAL_BATCH_SIZE).prefetch(buffer_size=AUTOTUNE)

    # --- Bắt đầu quá trình đánh giá ---
    evaluation_results = []
    output_filename = "teacher_models_evaluation_summary.csv"
    output_filepath = os.path.join(KAGGLE_OUTPUT_PATH, output_filename)
    n_classes = len(ALL_CLASSES)

    # Vòng lặp qua 5 Fold để tải và đánh giá từng mô hình
    for fold_number in range(1, N_SPLITS + 1):
        print(f"\\n---> Đang đánh giá Fold {fold_number}/{N_SPLITS}...")
        
        model_path = os.path.join(KAGGLE_OUTPUT_PATH, f'{MODEL_ID}_fold_{fold_number}.keras')
        
        if not os.path.exists(model_path):
            print(f"!!! Cảnh báo: Không tìm thấy file model cho Fold {fold_number} tại '{model_path}'. Bỏ qua fold này.")
            continue

        try:
            with strategy.scope():
                # Tải lại mô hình đã huấn luyện
                model = tf.keras.models.load_model(
                model_path,
                    custom_objects={
                        'MacroF1Score': MacroF1Score,
                        'FinalModel': FinalModel  # <-- THÊM DÒNG NÀY VÀO
                    }
                )
                # Biên dịch lại mô hình để đảm bảo các metrics được tính đúng
                model.compile(
                    loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
                    optimizer='adam', # Optimizer ở đây không quan trọng vì không huấn luyện lại
                    metrics=[
                        'accuracy', 
                        tf.keras.metrics.AUC(name='auc'), 
                        MacroF1Score(num_classes=n_classes, name='f1_macro')
                    ]
                )

            # Đánh giá mô hình trên tập Test đã được batch
            print(f"Bắt đầu tính toán các chỉ số cho Fold {fold_number}...")
            results = model.evaluate(test_ds_batched, return_dict=True, verbose=0)
            
            # Lưu kết quả của fold hiện tại
            fold_summary = {
                'Fold': fold_number,
                'Loss': results.get('loss', 0),
                'Accuracy': results.get('accuracy', 0),
                'AUC': results.get('auc', 0),
                'F1-Macro': results.get('f1_macro', 0)
            }
            evaluation_results.append(fold_summary)
            print(f"Kết quả Fold {fold_number}: {fold_summary}")

        except Exception as e:
            print(f"!!! Lỗi khi xử lý Fold {fold_number}: {e}")

    # --- Tổng hợp và Lưu kết quả ra file CSV ---
    if evaluation_results:
        results_df = pd.DataFrame(evaluation_results)
        
        # Tính toán giá trị trung bình và độ lệch chuẩn
        summary_stats = results_df.drop('Fold', axis=1).agg(['mean', 'std'])
        print("\\n\\n--- Thống kê tổng hợp (5 Folds) trên tập Test ---")
        print(summary_stats)
        
        # Tạo dòng tổng kết để thêm vào file CSV
        summary_stats_df = summary_stats.reset_index().rename(columns={'index': 'Fold'})
        
        # Ghép kết quả chi tiết và kết quả tổng hợp
        final_df_to_save = pd.concat([results_df, summary_stats_df], ignore_index=True)
        
        final_df_to_save.to_csv(output_filepath, index=False, float_format='%.4f')
        print(f"\\n--- HOÀN TẤT ---")
        print(f"Bảng kết quả tổng hợp đã được lưu tại: {output_filepath}")
        
        print("\\nNội dung file CSV:")
        print(final_df_to_save.to_string(index=False))
    else:
        print("\\nKhông có fold nào được đánh giá thành công. Không có file CSV nào được tạo.")

In [ ]:
# --- Bắt đầu quy trình vẽ đường cong ROC trung bình 5-Fold ---

print("--- Chuẩn bị dữ liệu Test cho việc vẽ ROC ---")

# 1. Tạo Test Dataset và trích xuất nhãn thật
TEST_TFREC_PATH = os.path.join(TFRECORD_OUTPUT_PATH, "test.tfrec")
if not os.path.exists(TEST_TFREC_PATH):
    print(f"Lỗi: Không tìm thấy file test.tfrec tại {TEST_TFREC_PATH}. Vui lòng chạy lại ô chuẩn bị dữ liệu.")
else:
    # Tạo dataset gốc để lấy nhãn
    test_ds_unbatched = tf.data.TFRecordDataset(TEST_TFREC_PATH)
    test_ds_unbatched = test_ds_unbatched.map(parse_tfrecord_fn, num_parallel_calls=AUTOTUNE)
    
    # Tạo dataset đã được batch để lấy dự đoán
    test_ds_batched = test_ds_unbatched.batch(GLOBAL_BATCH_SIZE).prefetch(buffer_size=AUTOTUNE)

    print("Đang trích xuất nhãn từ tập test...")
    y_test_onehot = np.concatenate([y.numpy() for x, y in test_ds_unbatched], axis=0)
    print(f"Đã trích xuất xong {len(y_test_onehot)} nhãn.")

    # --- Bắt đầu quá trình lấy dự đoán và vẽ biểu đồ ---
    y_test_binarized = y_test_onehot
    n_classes = y_test_binarized.shape[1]
    class_names = le.classes_
    
    all_y_preds_probs = []
    print("\\nĐang tải 5 model và lấy dự đoán từ 5 folds...")
    for fold_number in tqdm(range(1, N_SPLITS + 1), desc="Processing Folds for ROC"):
        model_path = os.path.join(KAGGLE_OUTPUT_PATH, f'{MODEL_ID}_fold_{fold_number}.keras')
        if not os.path.exists(model_path):
            continue
        
        with strategy.scope():
            model = tf.keras.models.load_model(
                model_path,
                custom_objects={
                    'MacroF1Score': MacroF1Score,
                    'FinalModel': FinalModel
                }
            )
        
        all_logits = []
        for images_batch, _ in test_ds_batched:
            batch_logits = model(images_batch, training=False)
            all_logits.append(batch_logits.numpy())
        
        y_pred_logits = np.concatenate(all_logits, axis=0)
        y_pred_probs = tf.nn.softmax(y_pred_logits).numpy()
        all_y_preds_probs.append(y_pred_probs)

    # --- Tính toán ROC và nội suy ---
    print("\\nĐang tính toán ROC và nội suy...")
    tprs_per_class = [[] for _ in range(n_classes)]
    aucs_per_class = [[] for _ in range(n_classes)]
    mean_fpr = np.linspace(0, 1, 100)

    for y_pred_probs in all_y_preds_probs:
        for i in range(n_classes):
            fpr, tpr, _ = roc_curve(y_test_binarized[:, i], y_pred_probs[:, i])
            interp_tpr = np.interp(mean_fpr, fpr, tpr)
            interp_tpr[0] = 0.0
            tprs_per_class[i].append(interp_tpr)
            aucs_per_class[i].append(sklearn_auc(fpr, tpr))

    # --- Vẽ biểu đồ ---
    print("Đang vẽ biểu đồ...")
    plt.figure(figsize=(12, 10))
    colors = plt.cm.get_cmap('tab10', n_classes)

    for i in range(n_classes):
        mean_tpr = np.mean(tprs_per_class[i], axis=0)
        mean_tpr[-1] = 1.0
        std_tpr = np.std(tprs_per_class[i], axis=0)
        mean_auc = np.mean(aucs_per_class[i])
        std_auc = np.std(aucs_per_class[i])

        plt.plot(mean_fpr, mean_tpr, color=colors(i),
                 label=f'Lớp: {class_names[i]} (AUC = {mean_auc:.2f} $\\pm$ {std_auc:.2f})',
                 lw=2)
        
        tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
        tprs_lower = np.maximum(mean_tpr - std_tpr, 0)
        plt.fill_between(mean_fpr, tprs_lower, tprs_upper, color=colors(i), alpha=.2)

    plt.plot([0, 1], [0, 1], 'k--', lw=2, label='Chance')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate', fontsize=14)
    plt.ylabel('True Positive Rate', fontsize=14)
    plt.title('Đường cong ROC trung bình (5-Fold Cross-Validation)', fontsize=16)
    plt.legend(loc="lower right", fontsize=12)
    plt.grid(True)
    
    output_filename = "roc_curve_5_folds_average.png"
    output_filepath = os.path.join(KAGGLE_OUTPUT_PATH, output_filename)
    plt.savefig(output_filepath, dpi=300)
    print(f"\\nHoàn tất! Biểu đồ đã được lưu tại: {output_filepath}")
    
    plt.show()

In [ ]:
# --- Bắt đầu quy trình phân tích Grad-CAM chi tiết cho 5 Folds ---

print("--- Chuẩn bị dữ liệu Test cho việc phân tích Grad-CAM ---")

# 1. Tạo Test Dataset từ file TFRecord đã được xử lý trước
# Lưu ý: Các biến TFRECORD_OUTPUT_PATH, parse_tfrecord_fn, GLOBAL_BATCH_SIZE, AUTOTUNE
# đã được định nghĩa ở các ô code phía trên.
TEST_TFREC_PATH = os.path.join(TFRECORD_OUTPUT_PATH, "test.tfrec")
if not os.path.exists(TEST_TFREC_PATH):
    print(f"Lỗi: Không tìm thấy file test.tfrec tại {TEST_TFREC_PATH}. Vui lòng chạy lại ô chuẩn bị dữ liệu.")
else:
    test_ds_batched = tf.data.TFRecordDataset(TEST_TFREC_PATH)
    test_ds_batched = test_ds_batched.map(parse_tfrecord_fn, num_parallel_calls=AUTOTUNE)
    test_ds_batched = test_ds_batched.batch(GLOBAL_BATCH_SIZE).prefetch(buffer_size=AUTOTUNE)

    # --- Định nghĩa các hàm cần thiết cho Grad-CAM ---
    target_names = le.classes_
    grad_cam_main_path = os.path.join(KAGGLE_OUTPUT_PATH, "grad_cam_detailed_analysis")
    os.makedirs(grad_cam_main_path, exist_ok=True)

    def find_last_conv_layer(model):
        for layer in reversed(model.layers):
            if isinstance(layer, tf.keras.layers.Conv2D):
                return layer
            if isinstance(layer, tf.keras.Model):
                return find_last_conv_layer(layer)
        return None

    @tf.function
    def get_grad_cam_batched(model, img_batch):
        if hasattr(model, 'base_model'):
            inp = model.base_model.input
            last_conv_layer = model.base_model.get_layer("top_conv")
            x = model.pooling(model.base_model.output)
            x = model.dense1(x)
            if hasattr(model, 'dense2'): # Thích ứng với head phức tạp hơn
                x = model.dense2(x)
            final_output = model.dense_output(x)
            grad_model = Model(inp, [last_conv_layer.output, final_output])
        else:
            last_conv_layer = find_last_conv_layer(model)
            grad_model = Model([model.inputs], [last_conv_layer.output, model.output])
        
        with tf.GradientTape() as tape:
            last_conv_layer_output_value, preds = grad_model(img_batch)
            pred_indices = tf.argmax(preds, axis=1)
            class_channels = tf.gather(preds, pred_indices, axis=1, batch_dims=1)

        grads = tape.gradient(class_channels, last_conv_layer_output_value)
        pooled_grads = tf.reduce_mean(grads, axis=(1, 2))
        heatmap_batch = tf.einsum('bhwc,bc->bhw', last_conv_layer_output_value, pooled_grads)
        heatmap_batch = tf.maximum(heatmap_batch, 0)
        max_vals = tf.reduce_max(heatmap_batch, axis=(1, 2), keepdims=True)
        heatmap_batch = heatmap_batch / (max_vals + tf.keras.backend.epsilon())
        return heatmap_batch, preds

    def run_grad_cam_analysis_final(model, model_name, output_base_path, test_dataset):
        print(f"\\n--- Bắt đầu phân tích cho mô hình: {model_name} ---")
        results_by_class = { name: {'correct_heatmaps': [], 'correct_confidences': [], 'correct_images': [],
                                    'incorrect_heatmaps': [], 'incorrect_confidences': [], 'incorrect_images': []}
                            for name in target_names }

        print("  - Xử lý các batch trên TPU...")
        for images_batch, labels_batch in tqdm(test_dataset, desc=f"Analyzing {model_name}"):
            heatmap_batch, preds_batch = get_grad_cam_batched(model, images_batch)
            y_pred_probs_batch = tf.nn.softmax(preds_batch).numpy()
            y_pred_batch = np.argmax(y_pred_probs_batch, axis=1)
            y_true_batch = np.argmax(labels_batch.numpy(), axis=1)

            for i in range(images_batch.shape[0]):
                y_pred, y_true = y_pred_batch[i], y_true_batch[i]
                true_class_name = target_names[y_true]
                
                if y_pred == y_true:
                    results_by_class[true_class_name]['correct_heatmaps'].append(heatmap_batch[i].numpy())
                    results_by_class[true_class_name]['correct_confidences'].append(y_pred_probs_batch[i, y_pred])
                    results_by_class[true_class_name]['correct_images'].append(images_batch[i].numpy())
                else:
                    results_by_class[true_class_name]['incorrect_heatmaps'].append(heatmap_batch[i].numpy())
                    results_by_class[true_class_name]['incorrect_confidences'].append(y_pred_probs_batch[i, y_pred])
                    results_by_class[true_class_name]['incorrect_images'].append(images_batch[i].numpy())

        print("  - Đang lưu ảnh Grad-CAM và ảnh spectrogram cho các mẫu tiêu biểu...")
        for class_name in target_names:
            class_output_path = os.path.join(output_base_path, class_name)
            os.makedirs(class_output_path, exist_ok=True)
            class_results = results_by_class[class_name]

            if class_results['correct_confidences']:
                best_idx = np.argmax(class_results['correct_confidences'])
                image = class_results['correct_images'][best_idx]
                heatmap = class_results['correct_heatmaps'][best_idx]
                overlay = overlay_grad_cam(image[:, :, 0], heatmap)
                gradcam_filename = f"{model_name}_{class_name}_exemplar_correct_gradcam.png"
                plt.imsave(os.path.join(class_output_path, gradcam_filename), overlay)
                spectrogram_filename = f"{model_name}_{class_name}_exemplar_correct_spectrogram.png"
                plt.imsave(os.path.join(class_output_path, spectrogram_filename), image[:, :, 0], cmap='viridis')

            if class_results['incorrect_confidences']:
                worst_idx = np.argmax(class_results['incorrect_confidences'])
                image = class_results['incorrect_images'][worst_idx]
                heatmap = class_results['incorrect_heatmaps'][worst_idx]
                overlay = overlay_grad_cam(image[:, :, 0], heatmap)
                gradcam_filename = f"{model_name}_{class_name}_exemplar_incorrect_gradcam.png"
                plt.imsave(os.path.join(class_output_path, gradcam_filename), overlay)
                spectrogram_filename = f"{model_name}_{class_name}_exemplar_incorrect_spectrogram.png"
                plt.imsave(os.path.join(class_output_path, spectrogram_filename), image[:, :, 0], cmap='viridis')

    # --- VÒNG LẶP CHÍNH ĐỂ PHÂN TÍCH 5 FOLDS ---
    teacher_models_main_path = os.path.join(grad_cam_main_path, "teacher_models")
    os.makedirs(teacher_models_main_path, exist_ok=True)

    for fold_number in range(1, N_SPLITS + 1):
        print(f"\\n---> Bắt đầu phân tích Grad-CAM cho Fold {fold_number}/{N_SPLITS}...")
        model_path = os.path.join(KAGGLE_OUTPUT_PATH, f'{MODEL_ID}_fold_{fold_number}.keras')
        
        if not os.path.exists(model_path):
            print(f"Bỏ qua Fold {fold_number}, không tìm thấy file: {model_path}")
            continue
        
        try:
            with strategy.scope():
                teacher_model = tf.keras.models.load_model(
                    model_path,
                    custom_objects={
                        'focal_loss_fixed': focal_loss_from_logits_optimized(alpha=alpha_weights_list),
                        'MacroF1Score': MacroF1Score 
                    }
                )
            
            model_name = f"fold_{fold_number}"
            fold_output_path = os.path.join(teacher_models_main_path, model_name)
            os.makedirs(fold_output_path, exist_ok=True)
            
            # Gọi hàm phân tích với dataset đã được chuẩn bị
            run_grad_cam_analysis_final(teacher_model, model_name, fold_output_path, test_ds_batched)
        
        except Exception as e:
            print(f"!!! Lỗi khi phân tích Grad-CAM cho Fold {fold_number}: {e}")

    print("\\n--- Toàn bộ quá trình phân tích Grad-CAM đã hoàn tất ---")

In [ ]:
# --- BƯỚC TỐI ƯU & CHUYỂN ĐỔI CẢ 5 MÔ HÌNH SANG TFLITE (POST-TRAINING) ---

print("--- Bắt đầu quy trình chuyển đổi 5-Fold sang TFLite ---")

# Kiểm tra xem quá trình huấn luyện đã hoàn tất và có đủ kết quả chưa
if 'fold_f1s' in locals() and len(fold_f1s) == N_SPLITS:
    
    # Lấy thông tin chia fold một lần để tái sử dụng
    skf_split = list(skf.split(train_val_df, y_labels_for_split, groups_for_split))

    # === BẮT ĐẦU VÒNG LẶP QUA 5 FOLDS ===
    for fold_index in range(N_SPLITS):
        fold_number = fold_index + 1
        print("=" * 60)
        print(f"--- Bắt đầu chuyển đổi cho Fold {fold_number} ---")
        
        # 1. Xác định đường dẫn cho fold hiện tại
        SAVED_MODEL_PATH = os.path.join(KAGGLE_OUTPUT_PATH, f'{MODEL_ID}_fold_{fold_number}')
        TFLITE_MODEL_PATH = os.path.join(KAGGLE_OUTPUT_PATH, f'model_fold_{fold_number}_quantized.tflite')

        if not os.path.exists(SAVED_MODEL_PATH):
            print(f"Lỗi: Không tìm thấy thư mục model tại '{SAVED_MODEL_PATH}'. Bỏ qua fold này.")
            continue

        # 2. Tạo Representative Dataset từ tập validation của CHÍNH FOLD NÀY
        print(f"Đang tạo representative dataset cho Fold {fold_number}...")
        _, val_indices = skf_split[fold_index]
        val_df_for_calib = train_val_df.iloc[val_indices]

        def representative_data_gen():
            for _, row in val_df_for_calib.sample(n=min(150, len(val_df_for_calib)), random_state=SEED).iterrows():
                spectrogram = np.load(row['filepath']).astype(np.float32)
                
                image_tensor = tf.convert_to_tensor(spectrogram)
                image_3d = tf.stack([image_tensor]*3, axis=-1)
                image_resized = tf.image.resize(image_3d, [INPUT_SHAPE[0], INPUT_SHAPE[1]])
                min_val = tf.reduce_min(image_resized)
                max_val = tf.reduce_max(image_resized)
                image_scaled_01 = (image_resized - min_val) / (max_val - min_val + 1e-7)
                image_scaled_255 = image_scaled_01 * 255.0
                image_preprocessed = preprocess_input(image_scaled_255)
                
                yield [tf.expand_dims(image_preprocessed, axis=0)]

        # 3. Chuyển đổi và Lượng tử hóa
        print(f"Đang chuyển đổi mô hình của Fold {fold_number}...")
        converter = tf.lite.TFLiteConverter.from_saved_model(SAVED_MODEL_PATH)
        converter.optimizations = [tf.lite.Optimize.DEFAULT]
        converter.representative_dataset = representative_data_gen
        converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
        
        tflite_quant_model = converter.convert()

        # 4. Lưu file TFLite
        with open(TFLITE_MODEL_PATH, 'wb') as f:
            f.write(tflite_quant_model)
        
        print(f"Đã lưu thành công model TFLite cho Fold {fold_number} tại: {TFLITE_MODEL_PATH}")
        print(f"Kích thước file: {len(tflite_quant_model) / (1024 * 1024):.2f} MB")

    print("=" * 60)
    print("\\n Hoàn tất chuyển đổi cho cả 5 mô hình!")

else:
    print("Lỗi: Không tìm thấy kết quả của 5 fold ('fold_f1s'). Vui lòng chạy ô huấn luyện trước khi chạy ô này.")